In [ ]:
# %pip install python-dotenv
# %pip install pandas
# %pip install requests
# %pip install pyshacl
# %pip install brickschema

In [2]:
import requests
import pandas as pd
from io import StringIO
from rdflib import Graph
import os
import pyshacl
import uuid
import json

Test query

In [ ]:
artifacts_sparql_endpoint = 'http://localhost:7400/repositories/dsm-mpc'

sparql_query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=sba:LoadForecastAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """

headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
}
auth = ("lchamari", "lchamari")

query_params = {
    'query': sparql_query,
    'infer': 'false' 
}

response = requests.get(artifacts_sparql_endpoint, params=query_params, headers=headers, auth=auth)

response_json = json.loads(response.text)
formatted_json = json.dumps(response_json, indent=4)
bindings = response_json["results"]["bindings"]
data = [
    {"Input": item["input"]["value"], "Timeseries Reference": item["tsref"]["value"]}
    for item in bindings
]
df = pd.DataFrame(data)
df

,Input,Timeseries Reference
0,https://w3id.org/b4b/schemes/algorithms/forc#W...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...
1,https://w3id.org/b4b/schemes/algorithms/forc#S...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...
2,https://w3id.org/b4b/schemes/algorithms/forc#O...,https://w3id.org/b4b/schemes/ext/ref-ext#Forec...
3,https://brickschema.org/schema/Brick#Building_...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
4,https://brickschema.org/schema/Brick#Outside_A...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
5,https://brickschema.org/schema/Brick#Wind_Spee...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...
6,https://brickschema.org/schema/Brick#Solar_Rad...,https://w3id.org/b4b/schemes/ext/ref-ext#Histo...


# Creating Shapes

In [4]:
def generate_shape_graphs(algorithm_sparql_query, folder_name):

    query_params = {
        'query': algorithm_sparql_query,
        'infer': 'false' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(artifacts_sparql_endpoint, params=query_params, headers=headers, auth=auth)
    
    prefixes = {
        'brick:': 'https://brickschema.org/schema/Brick#',
        'ref:': 'https://brickschema.org/schema/Brick/ref#',
        'ref-ext:': 'https://w3id.org/b4b/schemes/ext/ref-ext#',
        'sba:': 'https://w3id.org/b4b/schemes/algorithms/sba#', 
        'mpc:': 'https://w3id.org/b4b/schemes/algorithms/mpc#', 
        'forc:': 'https://w3id.org/b4b/schemes/algorithms/forc#', 
        'brick-ev': '<https://w3id.org/b4b/schemes/ext/brick-ev#>'
    }

    response_json = json.loads(response.text)
    bindings = response_json["results"]["bindings"]
    data = [
        {"Input": item["input"]["value"], "Timeseries Reference": item["tsref"]["value"]}
        for item in bindings
    ]
    df = pd.DataFrame(data)

    class_list = df['Input'].tolist()

    point_with_suffix_list = []
    for point in class_list:
        suffix = point.split('#')[0] + '#'
        point = point.split('#')[-1]

        for prefix, uri in prefixes.items():
            if uri == suffix:
                suffix = prefix
                point_with_suffix = f'{suffix}:{point}'
                point_with_suffix_list.append(point_with_suffix)
                break
  
    dplist = {}
    for index, row in df.iterrows():
        if row['Input'] and row['Timeseries Reference']:
            dplist[row['Input']] = row['Timeseries Reference']
    print(dplist)
    

    for key, value in dplist.items():
        for prefix, uri in prefixes.items():
            if uri in value:
                dplist[key] = value.replace(uri, prefix)
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/forc#', 'forc:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://brickschema.org/schema/Brick#', 'brick:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/sba#', 'sba:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/algorithms/mpc#', 'mpc:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://brickschema.org/schema/Brick/ref#', 'ref:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/ext/ref-ext#', 'ref-ext:'): value for key, value in dplist.items()}
    dplist = {key.replace('https://w3id.org/b4b/schemes/ext/brick-ev#', 'brick-ev:'): value for key, value in dplist.items()}

    dict = {
    "ref-ext:LastKnownValueTimeseriesReference": "ref-ext:hasLastKnownValueTimeseriesId",
    "ref-ext:HistoricalTimeseriesReference": "ref-ext:hasHistoricalTimeseriesId",
    "ref-ext:RealtimeTimeseriesReference": "ref-ext:hasRealtimeTimeseriesId",
    "ref-ext:ForecastTimeseriesReference": "ref-ext:hasForecastTimeseriesId",
    "ref:TimeseriesReference": "ref:hasTimeseriesId",
    }

    shape_graph_list = []
    class_names = []
    class_names_ = []
    for k in dplist:
        value_2 = dict[dplist[k]]
        shape_graph = f"""
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#>.
@prefix ref: <https://brickschema.org/schema/Brick/ref#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#> .
@prefix sba: <https://w3id.org/b4b/schemes/algorithms/sba#> .
@prefix mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> .
@prefix forc: <https://w3id.org/b4b/schemes/algorithms/forc#> .
@prefix brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#> .
@prefix : <urn:algorithm_shapes#> .

:AlgorithmInputShape_{uuid.uuid4()} 
    a sh:NodeShape ;
    sh:targetClass {k} ;
    sh:property :property_1;
.

:property_1
        sh:path ref:hasExternalReference ;
        sh:message "All datapoints need an external reference." ;
        sh:minCount 1 ;
        sh:node sh:NodeShape ;
        sh:property :property2, :property3 
.

:property2
            sh:path rdf:type ;
            sh:hasValue {dplist[k]} ;
.

:property3 
            sh:path {value_2} ;
            sh:minCount 1 ;
            sh:datatype xsd:string ;
            sh:message "All datapoints need a timeseries reference." ;
.

"""
        shape_graph_list.append(shape_graph)
        class_names.append(k)
        class_name = k.replace(":", "_")
        class_names_.append(class_name)
        

    i=0
    for g in shape_graph_list:
        with open(f"shacl/{folder_name}/shape_graph_{class_names_[i]}.ttl", "w") as f:
            f.write(g)
            i=i+1

    return df, dplist

In [5]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=sba:LoadForecastAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
df,dplist_LoadForecastAlgorithm = generate_shape_graphs(algorithm_sparql_query=query, folder_name="LoadForecastAlgorithm")
dplist_LoadForecastAlgorithm

{'https://w3id.org/b4b/schemes/algorithms/forc#WindSpeedForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference', 'https://w3id.org/b4b/schemes/algorithms/forc#SolarRadianceForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference', 'https://w3id.org/b4b/schemes/algorithms/forc#OutdoorTemperatureForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference', 'https://brickschema.org/schema/Brick#Building_Electrical_Meter': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference', 'https://brickschema.org/schema/Brick#Outside_Air_Temperature_Sensor': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference', 'https://brickschema.org/schema/Brick#Wind_Speed_Sensor': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference', 'https://brickschema.org/schema/Brick#Solar_Radiance_Sensor': 'https://w3id.org/b4b/schemes/ext/ref-ext#HistoricalTimeseriesReference'}


{'forc:WindSpeedForecast': 'ref-ext:ForecastTimeseriesReference',
 'forc:SolarRadianceForecast': 'ref-ext:ForecastTimeseriesReference',
 'forc:OutdoorTemperatureForecast': 'ref-ext:ForecastTimeseriesReference',
 'brick:Building_Electrical_Meter': 'ref-ext:HistoricalTimeseriesReference',
 'brick:Outside_Air_Temperature_Sensor': 'ref-ext:HistoricalTimeseriesReference',
 'brick:Wind_Speed_Sensor': 'ref-ext:HistoricalTimeseriesReference',
 'brick:Solar_Radiance_Sensor': 'ref-ext:HistoricalTimeseriesReference'}

In [6]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=mpc:DSM-MPC)
    FILTER(?p=mpc:hasInputDisturbanceForecast || ?p=mpc:hasInputRealtimeFeedback )
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
df, dplist_MPC = generate_shape_graphs(algorithm_sparql_query=query, folder_name="DSM-MPC")
dplist_MPC

{'https://w3id.org/b4b/schemes/algorithms/forc#BuildingLoadForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference', 'https://w3id.org/b4b/schemes/algorithms/forc#PVForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference', 'https://w3id.org/b4b/schemes/algorithms/forc#ElectricityPriceForecast': 'https://w3id.org/b4b/schemes/ext/ref-ext#ForecastTimeseriesReference'}


{'forc:BuildingLoadForecast': 'ref-ext:ForecastTimeseriesReference',
 'forc:PVForecast': 'ref-ext:ForecastTimeseriesReference',
 'forc:ElectricityPriceForecast': 'ref-ext:ForecastTimeseriesReference'}

In [203]:
query = """
PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

select ?input ?tsref where {
    ?s ?p ?input .
    FILTER(?s=sba:RealtimeFeedbackAlgorithm)
    FILTER(?p=sba:hasInputData)
    ?input ref:hasExternalReference ?arr .
    ?arr a ?tsref .
}
    """
df, dplist_RealtimeFeedbackAlgorithm = generate_shape_graphs(algorithm_sparql_query=query, folder_name="RealtimeFeedbackAlgorithm")
dplist_RealtimeFeedbackAlgorithm

{'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Charging_Station_ID': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Battery_Size': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_Time': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Departure_Time': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_Arrival_SOC': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#Charging_Session_Status': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#Vehicle_Full_Status': 'https://w3id.org/b4b/schemes/ext/ref-ext#RealtimeTimeseriesReference', 'https://w3id.org/b4b/schemes/ext/brick-ev#EV_El

{'brick-ev:EV_Charging_Station_ID': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Battery_Size': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Arrival_Time': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Departure_Time': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Arrival_SOC': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:Charging_Session_Status': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:Vehicle_Full_Status': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Electric_Energy_Meter': 'ref-ext:RealtimeTimeseriesReference',
 'brick-ev:EV_Charging_Session_Visual_Number': 'ref-ext:RealtimeTimeseriesReference'}

In [204]:
dplist_LoadForecastAlgorithm=list(dplist_LoadForecastAlgorithm.keys())
dplist_MPC=list(dplist_MPC.keys())
dplist_RealtimeFeedbackAlgorithm=list(dplist_RealtimeFeedbackAlgorithm.keys())

# Check

In [ ]:
def ask_query(building_sparql_endpoint, point_list):

    construct_query = """
    CONSTRUCT { ?s ?p ?o }
    WHERE { ?s ?p ?o }
    """
    query_params = {
        'query': construct_query,
        'infer': 'true' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

    if response.status_code == 200:
        response_text = response.text
        building2_data_graph= Graph()
        building2_data_graph.parse(data=response_text, format="turtle")
        print(f"Loaded {len(building2_data_graph)} tripples")

    else:
        print(f"Error: {response.status_code} - {response.reason}")


    status_dict = {}
    for point in point_list:
            ask_query = f"""
            PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
            PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
            PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
            PREFIX brick: <https://brickschema.org/schema/Brick#>
            PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
            PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
            PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

            ASK WHERE {{
                ?s a {point}
            }}
            """
            res = building2_data_graph.query(ask_query)
            staus=res.askAnswer
            status_dict[point] = staus
    df=pd.DataFrame(status_dict.items(), columns=['Data Point', 'Status'])
    df.sort_values(by=['Status'], ascending=False)
    return df

### Breda

In [ ]:
breda_sparql_endpoint='http://localhost:7400/repositories/breda'

In [276]:
df=ask_query(breda_sparql_endpoint, dplist_LoadForecastAlgorithm)
df

Loaded 605 tripples


,Data Point,Status
0,forc:WindSpeedForecast,True
1,forc:SolarRadianceForecast,True
2,forc:OutdoorTemperatureForecast,True
3,brick:Building_Electrical_Meter,True
4,brick:Outside_Air_Temperature_Sensor,True
5,brick:Wind_Speed_Sensor,True
6,brick:Solar_Radiance_Sensor,True


In [216]:
dplist_LoadForecastAlgorithm

['forc:WindSpeedForecast',
 'forc:SolarRadianceForecast',
 'forc:OutdoorTemperatureForecast',
 'brick:Building_Electrical_Meter',
 'brick:Outside_Air_Temperature_Sensor',
 'brick:Wind_Speed_Sensor',
 'brick:Solar_Radiance_Sensor']

In [217]:
df=ask_query(breda_sparql_endpoint, dplist_MPC)
df

Loaded 643 tripples


,Data Point,Status
0,forc:BuildingLoadForecast,False
1,forc:PVForecast,True
2,forc:ElectricityPriceForecast,True


In [218]:
df=ask_query(breda_sparql_endpoint, dplist_RealtimeFeedbackAlgorithm)
df

Loaded 643 tripples


,Data Point,Status
0,brick-ev:EV_Charging_Station_ID,True
1,brick-ev:EV_Battery_Size,True
2,brick-ev:EV_Arrival_Time,True
3,brick-ev:EV_Departure_Time,True
4,brick-ev:EV_Arrival_SOC,True
5,brick-ev:Charging_Session_Status,True
6,brick-ev:Vehicle_Full_Status,True
7,brick-ev:EV_Electric_Energy_Meter,True
8,brick-ev:EV_Charging_Session_Visual_Number,True


# Validate

In [236]:
def validate_graph(data_graph, shape_graph):
    shapes_file_format = "turtle"
    data_file_format = "turtle"

    res = pyshacl.validate(
        data_graph=data_graph,
        shacl_graph=shape_graph,
        data_graph_format=data_file_format,
        shacl_graph_format=shapes_file_format,
        inference="both",
        debug=False
    )
    return res

In [237]:
folder_names = ["DSM-MPC", "LoadForecastAlgorithm", "RealtimeFeedbackAlgorithm"]

In [ ]:
construct_query = """
CONSTRUCT { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""
query_params = {
    'query': construct_query,
    'infer': 'true' 
}
headers = {
"Content-Type": "application/sparql-query",
"Accept": "application/json"
}
auth = ("lchamari", "lchamari")

response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

if response.status_code == 200:
    response_text = response.text
    building_data_graph= Graph()
    building_data_graph.parse(data=response_text, format="turtle")
    print(f"Loaded {len(building_data_graph)} tripples")

else:
    print(f"Error: {response.status_code} - {response.reason}")

Loaded 605 tripples


In [242]:
def validate_artifact(folder_name):
    full_text = ""
    folder_path = f"shacl\{folder_name}"
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.ttl'):
            file_path = os.path.join(folder_path, file_name)
            temp_graph = Graph()
            temp_graph.parse(file_path, format='turtle')
            res = validate_graph(building_data_graph, temp_graph)
            conforms, results_graph, results_text = res
            full_text = full_text + file_name + "\n"
            full_text = full_text + results_text + "\n"
    with open(f"validation/building_1/{folder_name}/validation.txt", "w") as f:
        f.write(full_text)
    return full_text


In [243]:
validate_artifact("DSM-MPC")
validate_artifact("LoadForecastAlgorithm")
validate_artifact("RealtimeFeedbackAlgorithm")

'shape_graph_brick-ev_Charging_Session_Status.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Arrival_SOC.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Arrival_Time.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Battery_Size.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Charging_Session_Visual_Number.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Charging_Station_ID.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Departure_Time.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Electric_Energy_Meter.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_Vehicle_Full_Status.ttl\nValidation Report\nConforms: True\n\n'

### Building_v1

In [ ]:
building_sparql_endpoint = 'http://localhost:7400/repositories/building_v1'

In [ ]:
construct_query = """
CONSTRUCT { ?s ?p ?o }
WHERE { ?s ?p ?o }
"""
query_params = {
    'query': construct_query,
    'infer': 'true' 
}
headers = {
"Content-Type": "application/sparql-query",
"Accept": "application/json"
}
auth = ("lchamari", "lchamari")

response = requests.get(building_sparql_endpoint, params=query_params,  auth=auth)

if response.status_code == 200:
    response_text = response.text
    building_data_graph= Graph()
    building_data_graph.parse(data=response_text, format="turtle")
    print(f"Loaded {len(building_data_graph)} tripples")

else:
    print(f"Error: {response.status_code} - {response.reason}")

Loaded 572 tripples


In [246]:
def validate_artifact(folder_name):
    full_text = ""
    folder_path = f"shacl\{folder_name}"
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.ttl'):
            file_path = os.path.join(folder_path, file_name)
            temp_graph = Graph()
            temp_graph.parse(file_path, format='turtle')
            res = validate_graph(building_data_graph, temp_graph)
            conforms, results_graph, results_text = res
            full_text = full_text + file_name + "\n"
            full_text = full_text + results_text + "\n"
    with open(f"validation/building_v1/{folder_name}/validation.txt", "w") as f:
        f.write(full_text)
    return full_text


In [247]:
validate_artifact("DSM-MPC")
validate_artifact("LoadForecastAlgorithm")
validate_artifact("RealtimeFeedbackAlgorithm")

'shape_graph_brick-ev_Charging_Session_Status.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Arrival_SOC.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Arrival_Time.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Battery_Size.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Charging_Session_Visual_Number.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Charging_Station_ID.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Departure_Time.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_EV_Electric_Energy_Meter.ttl\nValidation Report\nConforms: True\n\nshape_graph_brick-ev_Vehicle_Full_Status.ttl\nValidation Report\nConforms: True\n\n'

# Configure

In [ ]:
def configure_algorithm(query, building_url):

    query_params = {
        'query': query,
        'infer': 'false' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(building_url, params=query_params,  auth=auth, headers=headers)

    if response.status_code == 200:
        response_json = response.json()
        bindings = response_json["results"]["bindings"]
        data = [
            {
                "WindSpeedForecast": item.get("WindSpeedForecast", {}).get("value", ""),
                "SolarRadianceForecast": item.get("SolarRadianceForecast", {}).get("value", ""),
                "OutdoorTemperatureForecast": item.get("OutdoorTemperatureForecast", {}).get("value", ""),
                "BuildingLoad": item.get("BuildingLoad", {}).get("value", "") ,
                "OutdoorTemperature": item.get("OutdoorTemperature", {}).get("value", "") ,
                "WindSpeed": item.get("WindSpeed", {}).get("value", "") ,
                "SolarRadiance": item.get("SolarRadiance", {}).get("value", ""),
            }
            for item in bindings
        ]
        df=pd.DataFrame(data)

    return df

##### Forecasting service

In [321]:
query=""" PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT ?WindSpeedForecast ?SolarRadianceForecast ?OutdoorTemperatureForecast ?BuildingLoad ?OutdoorTemperature ?WindSpeed ?SolarRadiance where { 
 
?windForc a  forc:WindSpeedForecast .
?windForc  ref:hasExternalReference ?windForcExt .
?windForcExt ref-ext:hasForecastTimeseriesId ?WindSpeedForecast .

?ghiForc a  forc:SolarRadianceForecast .
?ghiForc  ref:hasExternalReference ?ghiForcExt .
?ghiForcExt ref-ext:hasForecastTimeseriesId ?SolarRadianceForecast .

?tempForc a  forc:OutdoorTemperatureForecast .
?tempForc  ref:hasExternalReference ?tempForcExt .
?tempForcExt ref-ext:hasForecastTimeseriesId ?OutdoorTemperatureForecast .

?bLoad a  brick:Building_Electrical_Meter .
?bLoad  ref:hasExternalReference ?bLoadExt .
?bLoadExt ref-ext:hasHistoricalTimeseriesId ?BuildingLoad .

?temp a  brick:Outside_Air_Temperature_Sensor .
?temp  ref:hasExternalReference ?tempExt .
?tempExt ref-ext:hasHistoricalTimeseriesId ?OutdoorTemperature .

?windSp a  brick:Wind_Speed_Sensor .
?windSp  ref:hasExternalReference ?windSpExt .
?windSpExt ref-ext:hasHistoricalTimeseriesId ?WindSpeed .

?ghi a  brick:Solar_Radiance_Sensor .
?ghi  ref:hasExternalReference ?ghiExt .
?ghiExt ref-ext:hasHistoricalTimeseriesId ?SolarRadiance .
    
} """

In [332]:
df = configure_algorithm(query, breda_sparql_endpoint)
data = df.to_dict(orient='records')
lines = [f"{key}={value}" for key, value in data[0].items()]
print(lines)

file_path = "config/LoadForecastAlgorithm/forecast.env"

with open(file_path, "w") as file:
    file.write("\n".join(lines))

['WindSpeedForecast=3ede3817d96d7bb87865eaa7ac657860.Value', 'SolarRadianceForecast=3ede3817d96d7bb87865eaa7ac657860.Value', 'OutdoorTemperatureForecast=3ede3817d96d7bb87865eaa7ac657860.Value', 'BuildingLoad=f30ca6c6d2b3111dee9066951e270141.Value', 'OutdoorTemperature=a683b494a806af9a8da4d35ae99ba763.Value', 'WindSpeed=a683b494a806af9a8da4d35ae99ba763.Value', 'SolarRadiance=a683b494a806af9a8da4d35ae99ba763.Value']


##### Realtime Feedback

In [ ]:
def configure_feedback_algorithm(query, building_url):

    query_params = {
        'query': query,
        'infer': 'false' 
    }
    headers = {
    "Content-Type": "application/sparql-query",
    "Accept": "application/json"
    }
    auth = ("lchamari", "lchamari")

    response = requests.get(building_url, params=query_params,  auth=auth, headers=headers)

    if response.status_code == 200:
        response_json = response.json()
        bindings = response_json["results"]["bindings"]
        print(bindings)
    return bindings

In [348]:
query="""PREFIX ref-ext: <https://w3id.org/b4b/schemes/ext/ref-ext#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX brick-ev: <https://w3id.org/b4b/schemes/ext/brick-ev#>
PREFIX sba: <https://w3id.org/b4b/schemes/algorithms/sba#> 
PREFIX mpc: <https://w3id.org/b4b/schemes/algorithms/mpc#> 
PREFIX forc: <https://w3id.org/b4b/schemes/algorithms/forc#> 

SELECT DISTINCT ?chrgStId ?timeArrId ?timeDeptId ?arrivalSOCId ?evBatId ?vFullId ?chrgStatusId ?energyId ?visualNumId
where { 
    
?s a brick:Electric_Vehicle_Charging_Port .
#?s eva:physical_reference "2" .
?s brick:hasPoint ?chrgSt, ?timeArr, ?timeDept, ?arrivalSOC, ?evBat, ?vFull, ?chrgStatus, ?energy, ?visualNum .
    
?chrgSt a brick-ev:EV_Charging_Station_ID .
?chrgSt  ref:hasExternalReference ?chrgStExt .
?chrgStExt ref-ext:hasRealtimeTimeseriesId ?chrgStId .
        
?timeArr a brick-ev:EV_Arrival_Time .
?timeArr  ref:hasExternalReference ?timeArrExt .
?timeArrExt ref-ext:hasRealtimeTimeseriesId ?timeArrId .
    
?timeDept a brick-ev:EV_Departure_Time .
?timeDept  ref:hasExternalReference ?timeDeptExt .
?timeDeptExt ref-ext:hasRealtimeTimeseriesId ?timeDeptId .
    
?arrivalSOC a brick-ev:EV_Arrival_SOC .
?arrivalSOC  ref:hasExternalReference ?arrivalSOCExt .
?arrivalSOCExt ref-ext:hasRealtimeTimeseriesId ?arrivalSOCId .

?evBat a brick-ev:EV_Battery_Size .
?evBat  ref:hasExternalReference ?evBatExt .
?evBatExt ref-ext:hasRealtimeTimeseriesId ?evBatId .

?vFull a brick-ev:Vehicle_Full_Status .
?vFull  ref:hasExternalReference ?vFullExt .
?vFullExt ref-ext:hasRealtimeTimeseriesId ?vFullId .

?vFull a brick-ev:Vehicle_Full_Status .
?vFull  ref:hasExternalReference ?vFullExt .
?vFullExt ref-ext:hasRealtimeTimeseriesId ?vFullId .

?chrgStatus a brick-ev:Charging_Session_Status .
?chrgStatus  ref:hasExternalReference ?chrgStatusExt .
?chrgStatusExt ref-ext:hasRealtimeTimeseriesId ?chrgStatusId .
        
?energy a brick-ev:EV_Electric_Energy_Meter .
?energy  ref:hasExternalReference ?energyExt .
?energyExt ref-ext:hasRealtimeTimeseriesId ?energyId .

?visualNum a brick-ev:EV_Charging_Session_Visual_Number .
?visualNum  ref:hasExternalReference ?visualNumExt .
?visualNumExt ref-ext:hasRealtimeTimeseriesId ?visualNumId .

}"""

In [366]:
data = configure_feedback_algorithm(query, breda_sparql_endpoint)
lines1 = [f"{key}1={value['value']}" for key, value in data[0].items()]
lines2 = [f"{key}2={value['value']}" for key, value in data[1].items()]
lines3 = [f"{key}3={value['value']}" for key, value in data[2].items()]
lines4 = [f"{key}4={value['value']}" for key, value in data[3].items()]

file_path1 = "config/RealtimeFeedbackAlgorithm/feedback_1.env"
file_path2 = "config/RealtimeFeedbackAlgorithm/feedback_2.env"
file_path3 = "config/RealtimeFeedbackAlgorithm/feedback_3.env"
file_path4 = "config/RealtimeFeedbackAlgorithm/feedback_4.env"

with open(file_path1, "w") as file:
    file.write("\n".join(lines1))
with open(file_path2, "w") as file:
    file.write("\n".join(lines2))
with open(file_path3, "w") as file:
    file.write("\n".join(lines3))
with open(file_path4, "w") as file:
    file.write("\n".join(lines4))

input_files = ["config/RealtimeFeedbackAlgorithm/feedback_1.env", "config/RealtimeFeedbackAlgorithm/feedback_2.env", 
               "config/RealtimeFeedbackAlgorithm/feedback_3.env", "config/RealtimeFeedbackAlgorithm/feedback_4.env"]

output_file = "config/RealtimeFeedbackAlgorithm/feedback.env"

with open(output_file, "w") as outfile:
    for file in input_files:
        with open(file, "r") as infile:
            outfile.write(infile.read()) 
            outfile.write("\n")


[{'chrgStId': {'type': 'literal', 'value': '1'}, 'timeArrId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'timeDeptId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'arrivalSOCId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'evBatId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'vFullId': {'type': 'literal', 'value': '759ca9cc4f3bf8f552084d67f25d4589.vehicle_full_date'}, 'chrgStatusId': {'type': 'literal', 'value': '759ca9cc4f3bf8f552084d67f25d4589.status'}, 'energyId': {'type': 'literal', 'value': '759ca9cc4f3bf8f552084d67f25d45899.kwh'}, 'visualNumId': {'type': 'literal', 'value': '759ca9cc4f3bf8f552084d67f25d4589.visual_number'}}, {'chrgStId': {'type': 'literal', 'value': '2'}, 'timeArrId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'timeDeptId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'arrivalSOCId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'evBatId': {'type': 'literal', 'value': 'evse/welkomkropman'}, 'vFullId': {'typ